In [2]:
from IPython.display import clear_output
!pip install chart-studio
clear_output()
from sklearn.impute import KNNImputer
import seaborn as sns
import xgboost as xgb
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
from datetime import datetime as dt
import itertools
from tqdm import tqdm
from scipy.stats import norm, t
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler
import lightgbm as lgb
from sklearn.linear_model import LinearRegression
import gc
from sklearn.model_selection import StratifiedKFold,KFold
from sklearn import metrics
import os
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
pd.set_option('display.max_columns', None)
plt.style.use('fivethirtyeight') 
from pylab import rcParams
from plotly import tools
import chart_studio.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff
import statsmodels.api as sm
from numpy.random import normal, seed
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.arima_model import ARIMA
import math
from sklearn.metrics import mean_squared_error
from catboost import Pool, CatBoostRegressor
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from keras.callbacks import EarlyStopping
import os 
import random
# import numpy as np 
from catboost import Pool, CatBoostRegressor

DEFAULT_RANDOM_SEED = 2021

def seedBasic(seed=DEFAULT_RANDOM_SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
# tensorflow random seed 
import tensorflow as tf 
def seedTF(seed=DEFAULT_RANDOM_SEED):
    tf.random.set_seed(seed)
    
# torch random seed
import torch
def seedTorch(seed=DEFAULT_RANDOM_SEED):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
      
# basic + tensorflow + torch 
def seedEverything(seed=DEFAULT_RANDOM_SEED):
    seedBasic(seed)
    seedTF(seed)
    seedTorch(seed)
seedEverything(42)
seed = 42


In [3]:
train_ori = pd.read_csv('/kaggle/input/airquoi/train.csv',parse_dates=['date'])
test_ori = pd.read_csv('/kaggle/input/airquoi/test.csv',parse_dates=['date'])
ss =pd.read_csv('/kaggle/input/airquoi/SampleSubmission.csv')

In [4]:
#

# Helper functions

In [5]:
def cust_metric(y_pred, ds):
    y_true = ds.get_label()
    sq_errs = abs(y_true - y_pred)
    avg_per_id = pd.Series(sq_errs).groupby(ds.site_code).mean()
#     print(avg_per_id)
    return 'avg_worst_by_id', avg_per_id.max(), False

def _get_X_Y_DF_from_CV(train_X, train_Y, train_index, validation_index):
        X_train, X_validation = (
            train_X.iloc[train_index],
            train_X.iloc[validation_index],
        )
        y_train, y_validation = (
            train_Y.iloc[train_index],
            train_Y.iloc[validation_index],
        )
        return X_train, X_validation, y_train, y_validation
def cust_score(y_predicted):
    l = []
    a = train.site_latitude.unique()
    for i in a:
        y_true = train.loc[train.site_latitude == i,'pm2_5']
        y_hat = y_predicted[y_true.index]
        l.append(metrics.mean_absolute_error(y_true,y_hat))
    return pd.DataFrame({'site_latitude':a,
                        'err': np.array(l)})
def preds_to_sub(test,preds,name = None,log = False):
    if log :
        test['pm2_5'] = np.exp(preds)
    else:
        test['pm2_5'] = preds
    sub = test[['ID','pm2_5']]
    if name:
        sub.to_csv(name+'.csv', index = False)
    return sub


# Preprocessing

In [6]:
def preprocess(train,test,scale = True,target_encode = False,
               include_date = True,remove_nans = True, remove_multi = True):
    le = LabelEncoder()
    train['Train']=1
    test['Train']=0
    all_data=pd.concat([train,test])
    if include_date:
        all_data['month'] = all_data.date.dt.month
        all_data['year'] = all_data.date.dt.year
        all_data['day'] = all_data.date.dt.day
        all_data['week'] = all_data.date.dt.week
        all_data['dow'] = all_data.date.dt.dayofweek
        all_data['woy'] = all_data.date.dt.weekofyear
    all_data[['site_latitude','site_longitude']] =  all_data[['site_latitude','site_longitude']] .round(5)
    all_data['device'] = le.fit_transform(all_data['device'].values.reshape(-1,1))
    le = LabelEncoder()
    all_data['site_code'] = le.fit_transform(all_data.site_latitude.astype('str').values.reshape(-1,1))
    train = all_data.loc[all_data.Train == 1].reset_index(drop = True)
    test = all_data.loc[all_data.Train == 0].reset_index(drop = True)
    if target_encode:
        train,test = target_encode(train,test,target = 'pm2_5',groupss = 'site_code')
    if remove_nans:
        tr,te = cust_imputer(train.copy(), test.copy())
        all_data = pd.concat([tr,te]).reset_index(drop = True)
        s_codes = all_data.site_code.unique()
        distances = np.zeros((s_codes.shape[0],s_codes.shape[0]))
        for i in np.sort(s_codes):
            for j in np.sort(s_codes):
                distances[i,j] =  get_dist(all_data, i,j)
        distances = pd.DataFrame(index = np.sort(s_codes), columns = np.sort(s_codes),data = distances)
        SO2_cols = ['SulphurDioxide_SO2_column_number_density',
                    'SulphurDioxide_SO2_column_number_density_amf',
               'SulphurDioxide_SO2_slant_column_number_density',
               'SulphurDioxide_cloud_fraction',
                'SulphurDioxide_sensor_azimuth_angle',
               'SulphurDioxide_sensor_zenith_angle',
               'SulphurDioxide_solar_azimuth_angle',
               'SulphurDioxide_solar_zenith_angle',
               'SulphurDioxide_SO2_column_number_density_15km']
        CO_cols = ['CarbonMonoxide_CO_column_number_density',
               'CarbonMonoxide_H2O_column_number_density',
               'CarbonMonoxide_cloud_height', 'CarbonMonoxide_sensor_altitude',
               'CarbonMonoxide_sensor_azimuth_angle',
               'CarbonMonoxide_sensor_zenith_angle',
               'CarbonMonoxide_solar_azimuth_angle',
               'CarbonMonoxide_solar_zenith_angle']
        NO2_cols = ['NitrogenDioxide_NO2_column_number_density',
               'NitrogenDioxide_tropospheric_NO2_column_number_density',
               'NitrogenDioxide_stratospheric_NO2_column_number_density',
               'NitrogenDioxide_NO2_slant_column_number_density',
               'NitrogenDioxide_tropopause_pressure',
               'NitrogenDioxide_absorbing_aerosol_index',
               'NitrogenDioxide_cloud_fraction', 'NitrogenDioxide_sensor_altitude',
               'NitrogenDioxide_sensor_azimuth_angle',
               'NitrogenDioxide_sensor_zenith_angle',
               'NitrogenDioxide_solar_azimuth_angle',
               'NitrogenDioxide_solar_zenith_angle']
        HCHO_cols = ['Formaldehyde_tropospheric_HCHO_column_number_density',
               'Formaldehyde_tropospheric_HCHO_column_number_density_amf',
               'Formaldehyde_HCHO_slant_column_number_density',
               'Formaldehyde_cloud_fraction', 'Formaldehyde_solar_zenith_angle',
               'Formaldehyde_solar_azimuth_angle', 'Formaldehyde_sensor_zenith_angle',
               'Formaldehyde_sensor_azimuth_angle']
        UV_cols = ['UvAerosolIndex_absorbing_aerosol_index',
               'UvAerosolIndex_sensor_altitude', 'UvAerosolIndex_sensor_azimuth_angle',
               'UvAerosolIndex_sensor_zenith_angle',
               'UvAerosolIndex_solar_azimuth_angle',
               'UvAerosolIndex_solar_zenith_angle']
        O3_cols = ['Ozone_O3_column_number_density_amf',
               'Ozone_O3_slant_column_number_density',
               'Ozone_O3_effective_temperature', 'Ozone_cloud_fraction',
               'Ozone_sensor_azimuth_angle', 'Ozone_sensor_zenith_angle',
               'Ozone_solar_azimuth_angle', 'Ozone_solar_zenith_angle']
        cloud_cols = ['Cloud_cloud_fraction', 'Cloud_cloud_top_pressure',
               'Cloud_cloud_top_height', 'Cloud_cloud_base_pressure',
               'Cloud_cloud_base_height', 'Cloud_cloud_optical_depth',
               'Cloud_surface_albedo', 'Cloud_sensor_azimuth_angle',
               'Cloud_sensor_zenith_angle', 'Cloud_solar_azimuth_angle',
               'Cloud_solar_zenith_angle']
        target = ['pm2_5']
        all_cols = SO2_cols+cloud_cols+O3_cols+UV_cols+HCHO_cols+NO2_cols+CO_cols
        other_cols = ['pm2_5', 'Train', 'month', 'year', 'day',
               'week', 'dow', 'woy', 'rel_date']
        other_cols_test = ['Train', 'month', 'year', 'day',
               'week', 'dow', 'woy', 'rel_date']
        for col in tqdm(all_cols):
            for s_co in s_codes:
                all_data = distance_w_impute(col,s_co,all_data,distances)
        test = all_data[all_data.ID.isin(test_ori.ID)].drop_duplicates().reset_index(drop = True)
        train = all_data[all_data.ID.isin(train_ori.ID)].drop_duplicates().reset_index(drop = True)
        kn = KNNImputer(n_neighbors=5)
        cols = train.drop('pm2_5',axis = 1).select_dtypes(include = ['float','int']).columns
        train[cols] = kn.fit_transform(train[cols])
        test[cols] = kn.transform(test[cols])
        if scale:
            rs= RobustScaler()
            train[cols] = rs.fit_transform(train[cols])
            test[cols] = rs.transform(test[cols])
    if remove_multi:
        cols = train.select_dtypes(include = ['int','float']).columns
        l_drop = []
        for col in cols:
            df = train.pivot(index = 'date',columns = 'site_code',values = col)
            if df.corr().mean().mean()>0.99:
                l_drop.append(col)
        train = train.drop([l for l  in l_drop[1:] if l not in other_cols],axis = 1 )
        test = test.drop([l for l  in l_drop[1:] if l not in other_cols],axis = 1 )
    return train.sort_index(), test.sort_index()
def get_dist(all_data, i,j):
    return np.sqrt((all_data.loc[all_data.site_code == i,'site_latitude'].unique()[0]- all_data.loc[all_data.site_code == j,'site_latitude'].unique()[0])**2 + (all_data.loc[all_data.site_code == i,'site_longitude'].unique()[0]- all_data.loc[all_data.site_code == j,'site_longitude'].unique()[0])**2)

def distance_w_impute(col,s_co,all_data,distances):
    ind = all_data.loc[(all_data.site_code == s_co)& (all_data[col].isna()),'date'].values
    df3 = all_data.loc[(all_data.site_code != s_co)& (all_data.date.isin(ind)),['date',col,'site_code']]
    df3['weights'] = df3['site_code'].map(distances[s_co]**(-1))
    df3[col] = df3[col]*df3['weights']
    maps = df3.groupby('date').apply(lambda x : x[col].sum()/(x['weights'].sum()))
    all_data.loc[(all_data.site_code == s_co)& (all_data[col].isna()),col] = all_data.loc[(all_data.site_code == s_co)& (all_data[col].isna()),'date'].map(maps)
    return all_data
def cust_imputer(train, test):
    all_data = pd.concat([train,test]).reset_index(drop = True)
    sensor_az_ang = ['SulphurDioxide_sensor_azimuth_angle','CarbonMonoxide_sensor_azimuth_angle','NitrogenDioxide_sensor_azimuth_angle',
          'Formaldehyde_sensor_azimuth_angle','UvAerosolIndex_sensor_azimuth_angle','Ozone_sensor_azimuth_angle',
         'Cloud_sensor_azimuth_angle']
    sensor_zen_ang = ['SulphurDioxide_sensor_zenith_angle','CarbonMonoxide_sensor_zenith_angle','NitrogenDioxide_sensor_zenith_angle',
          'Formaldehyde_sensor_zenith_angle','UvAerosolIndex_sensor_zenith_angle','Ozone_sensor_zenith_angle',
         'Cloud_sensor_zenith_angle']
    solar_az_ang = ['SulphurDioxide_solar_azimuth_angle','CarbonMonoxide_solar_azimuth_angle','NitrogenDioxide_solar_azimuth_angle',
          'Formaldehyde_solar_azimuth_angle','UvAerosolIndex_solar_azimuth_angle','Ozone_solar_azimuth_angle',
         'Cloud_solar_azimuth_angle']
    solar_zen_ang = ['SulphurDioxide_solar_zenith_angle','CarbonMonoxide_solar_zenith_angle','NitrogenDioxide_solar_zenith_angle',
          'Formaldehyde_solar_zenith_angle','UvAerosolIndex_solar_zenith_angle','Ozone_solar_zenith_angle',
         'Cloud_solar_zenith_angle']
    X =  all_data[sensor_az_ang]
    imputer = KNNImputer(n_neighbors=5)
    all_data.loc[X.isna().sum(axis = 1)<7,sensor_az_ang] = imputer.fit_transform(X[X.isna().sum(axis = 1)<7]) 
    X =  all_data[sensor_zen_ang]
    imputer = KNNImputer(n_neighbors=5)
    all_data.loc[X.isna().sum(axis = 1)<7,sensor_zen_ang] = imputer.fit_transform(X[X.isna().sum(axis = 1)<7])
    X =  all_data[solar_az_ang]
    imputer = KNNImputer(n_neighbors=5)
    all_data.loc[X.isna().sum(axis = 1)<7,solar_az_ang] = imputer.fit_transform(X[X.isna().sum(axis = 1)<7]) 
    X =  all_data[solar_zen_ang]
    imputer = KNNImputer(n_neighbors=5)
    all_data.loc[X.isna().sum(axis = 1)<7,solar_zen_ang] = imputer.fit_transform(X[X.isna().sum(axis = 1)<7])
    return all_data.loc[:train.shape[0]], all_data.loc[train.shape[0]:]


# Models

In [64]:
def train_keras(baseline_model,train,features,target,test_data,kf,split_by):
    inp = train[features].shape[1]
    cats = train[split_by].values
    X_scaled = train[features].values
    Y = train[target].values
    
    y_oof = np.zeros(shape=(len(X_scaled),1))
    y_predicted = np.zeros(shape=(len(test_data),1))
    cv_scores = []
    test_data = test_data[features]
    models = []
    for i, (train_index, test_index) in enumerate(kf.split(X_scaled, cats)):
        print(' keras kfold: {}  of  {} : '.format(i+1, K_FOLDS ))
        X_train, X_valid = X_scaled[train_index], X_scaled[test_index]
        y_train, y_valid = Y[train_index], Y[test_index]
        my_model = baseline_model(inp)
        my_model.fit(X_train, y_train,
                     validation_data=(X_valid, y_valid),
                     epochs=100,
                     batch_size = 64,
                     callbacks=[EarlyStopping(patience=20)],
                     verbose=0)
        
        models.append(my_model)
        y_oof[test_index] = my_model.predict(X_valid)
        y_predicted += my_model.predict(test_data.values) 
        del my_model
        gc.collect()
        cv_oof_score = metrics.mean_absolute_error(y_valid, y_oof[test_index])
        cv_scores.append(cv_oof_score)
        print(f"CV OOF Score for fold {i+1} is {cv_oof_score}")

#         del validation_index, X_validation, y_validation
#         gc.collect()

    y_predicted /= K_FOLDS
    oof_score = round(metrics.mean_absolute_error(Y, y_oof), 5)
    avg_cv_scores = round(sum(cv_scores) / len(cv_scores), 5)
    std_cv_scores = round(np.array(cv_scores).std(), 5)
    return y_predicted,models,y_oof,oof_score 
def train_lgb(X,features,target,test_data,params,kf,split_by):
    features_importance= pd.DataFrame({'Feature':[], 'Importance':[]})
    models =[]
    train_X = X[features]
    train_Y = X[target]
    split_by = X[split_by]
    test_data = test_data[features]
    test_X = test_data.copy()
    print(f"Shape of train_X : {train_X.shape}, test_X: {test_X.shape}, train_Y: {train_Y.shape}")
    
    predictors = list(train_X.columns)
    # print(f"List of features to be used {list(predictors)}")

    # Selecting n_splits to be 3, since class 42 has 
    # just 3 instances
#     kf = KFold(random_state=seed_lgb,n_splits=K_FOLDS, shuffle=shuffle_lgb)
#     kf = StratifiedKFold(random_state=seed_lgb,n_splits=K_FOLDS, shuffle=shuffle_lgb)
    y_oof_lgb = np.zeros(shape=(len(train_X),))
    y_predicted_lgb = np.zeros(shape=(len(test_X),))
    cv_scores = []
    fold = 0
    n_folds = kf.get_n_splits()
    for train_index, validation_index in kf.split(X=train_X, y=split_by):
        fold += 1
        print(f"fold {fold} of {n_folds}")

        X_train, X_validation, y_train, y_validation = _get_X_Y_DF_from_CV(
            train_X, train_Y, train_index, validation_index
        )

        lgb_train = lgb.Dataset(X_train, y_train)
        lgb_eval = lgb.Dataset(X_validation, y_validation, reference=lgb_train)

        model = lgb.train(
            params,
            lgb_train,
            valid_sets=[lgb_train, lgb_eval],
            verbose_eval=-1,
#             early_stopping_rounds=EARLY_STOPPING_ROUNDS,
#             num_boost_round=N_ESTIMATORS,
            feature_name=predictors,
            categorical_feature="auto",
        )
        del lgb_train, lgb_eval, train_index, X_train, y_train
        gc.collect()

        y_oof_lgb[validation_index] = model.predict(
            X_validation, num_iteration=model.best_iteration
        )

        y_predicted_lgb += model.predict(
            test_data.values, num_iteration=model.best_iteration
        )
        fold_importance_df= pd.DataFrame({'Feature':[], 'Importance':[]})
        fold_importance_df['Feature']= predictors
        fold_importance_df['Importance']= model.feature_importance()
        fold_importance_df["fold"] = fold + 1
        features_importance = pd.concat([features_importance, fold_importance_df], axis=0)
        models.append(model)

        best_iteration = model.best_iteration
        print(f"Best number of iterations for fold {fold} is: {best_iteration}")

        cv_oof_score = metrics.mean_absolute_error(y_validation, y_oof_lgb[validation_index])
        cv_scores.append(cv_oof_score)
        print(f"CV OOF Score for fold {fold} is {cv_oof_score}")

        del validation_index, X_validation, y_validation
        gc.collect()

    y_predicted_lgb /= n_folds
    oof_score = round(metrics.mean_absolute_error(train_Y, y_oof_lgb), 5)
    avg_cv_scores = round(sum(cv_scores) / len(cv_scores), 5)
    std_cv_scores = round(np.array(cv_scores).std(), 5)
    return y_predicted_lgb,models,y_oof_lgb,oof_score,features_importance
def train_catbo(train_X,features, target,test_X,params,kf,split_by):
    y_oof = np.zeros(shape=(len(train_X),))
    y_predicted = np.zeros(shape=(len(test_X),))
    train_Y= train_X[target]
    split_by = train_X[split_by]
    train_X = train_X[features]
    test_X = test_X[features]
    cv_scores = []
    models = []
    fold = 0
    n_folds = kf.get_n_splits()
    for train_index, validation_index in kf.split(X=train_X, y=split_by):
        fold += 1
        print(f"fold {fold} of {n_folds}")
        X_train, X_validation, y_train, y_validation = _get_X_Y_DF_from_CV(
            train_X, train_Y, train_index, validation_index
        )
        train_pool = Pool(data=X_train, label=y_train)
        eval_pool = Pool(data=X_validation, label=y_validation.values) 
        model = CatBoostRegressor(**params)
        model.fit(train_pool,plot=True,eval_set=eval_pool)
        del train_index, X_train, y_train
        gc.collect()
        models.append(model)
        y_oof[validation_index] = model.predict(
            X_validation )

        y_predicted += model.predict(
            test_X.values
        )
        cv_oof_score = metrics.mean_absolute_error(y_validation, y_oof[validation_index])
        cv_scores.append(cv_oof_score)
        print(f"CV OOF Score for fold {fold} is {cv_oof_score}")

        del validation_index, X_validation, y_validation
        gc.collect()

    y_predicted /= n_folds
    oof_score = round(metrics.mean_absolute_error(train_Y, y_oof), 5)
    avg_cv_scores = round(sum(cv_scores) / len(cv_scores), 5)
    std_cv_scores = round(np.array(cv_scores).std(), 5)
    return y_predicted,models,y_oof,oof_score
def train_xgb(X,features,target,test_data,params,kf,split_by,num_iter=1500,es = 100,ve = 0):
#     features = X.columns
    y = train[target]
    split_by = train[split_by]
    X = X[features].values
    y_oof = np.zeros(shape=(len(X),))
    y_predicted = np.zeros(shape=(len(test_data), ))
    test_data = test_data[features]
    cv_scores = []
    models = []
    for i, (train_index, test_index) in enumerate(kf.split(X, split_by)):
        print(' xgb kfold: {}  of  {} : '.format(i+1, K_FOLDS ))
        X_train, X_valid = X[train_index], X[test_index]
        y_train, y_valid = y[train_index], y[test_index]
        d_train = xgb.DMatrix(X_train, y_train) 
        d_valid = xgb.DMatrix(X_valid, y_valid) 
        watchlist = [(d_train, 'train'), (d_valid, 'valid')]
        xgb_model = xgb.train(params, d_train, num_iter, watchlist,
                              early_stopping_rounds=es, 
                            verbose_eval=ve)
        models.append(xgb_model)
        y_oof[test_index] = xgb_model.predict(xgb.DMatrix(X_valid), 
                            ntree_limit=xgb_model.best_ntree_limit)
        y_predicted += xgb_model.predict(xgb.DMatrix(test_data[features].values), 
                            ntree_limit=xgb_model.best_ntree_limit) 
        
        cv_oof_score = metrics.mean_absolute_error(y_valid, y_oof[test_index])
        cv_scores.append(cv_oof_score)
        print(f"CV OOF Score for fold {i+1} is {cv_oof_score}")

    y_predicted /= K_FOLDS
    oof_score = round(metrics.mean_absolute_error(y, y_oof), 5)
    avg_cv_scores = round(sum(cv_scores) / len(cv_scores), 5)
    std_cv_scores = round(np.array(cv_scores).std(), 5)
    return y_predicted,models,y_oof,oof_score 


In [21]:
# kf

NameError: name 'kf' is not defined

In [8]:
train,test = preprocess(train_ori,test_ori)
clear_output()

In [9]:
drop_cols = ['ID','date','Train','pm2_5','site_latitudepm2_5enc7',
       'site_latitudepm2_5enc8', 'site_latitudepm2_5enc6',
       'site_latitudepm2_5enc5', 'site_latitudepm2_5enc4',
       'site_latitudepm2_5enc', 'site_latitudepm2_5enc1',
       'site_latitudepm2_5enc2', 'site_latitudepm2_5enc3','preds']
features  = [d for d in test.columns if d  not in drop_cols]

In [ ]:
SEED = 42

In [23]:
lgb_params = {
        "objective": "regression",
        "boosting_type": "gbdt",
        "learning_rate": 0.1,
        "n_jobs": 4,
        "seed": SEED,
        "max_depth": 3,
    #     "num_leaves":10,
        "force_col_wise":True,
    #     'min_data_in_leaf' : 5,
    #     'feature_fraction':0.5,
    #     'bagging_fraction':0.8,
    #     "max_bin": 255,
    #     'reg_lambda': 0.1,  # L1 regularization term on weights
    #     'reg_lambda': 10,
        "metric": "MAE",
        "verbose": -1,
    }
params_cat = {'iterations':5000,
        'learning_rate':0.025,
#         'random_strength':0.1,
        'early_stopping_rounds':30, 
        'depth':3,
        'loss_function':'RMSE',
        'eval_metric':'MAE',
        'verbose' : 100,
#         'leaf_estimation_method':'Newton'
             }
params_xgb = {"objective":"reg:squarederror",'learning_rate': 0.1,
           'max_depth': 3}#, 'subsample': 0.9,
#           'colsample_bytree': 0.9}
   
params={'params_lgb' : lgb_params,
      'params_xgb': params_xgb,
      'params_cat':params_cat}

In [24]:
K_FOLDS = 5
kf = KFold(n_splits = K_FOLDS, shuffle=False,random_state = None)

train_catbo(train,features, target,test,params['params_cat'],kf,target)

fold 1 of 5


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 21.0895256	test: 20.6929575	best: 20.6929575 (0)	total: 5.25ms	remaining: 26.2s
100:	learn: 16.1070486	test: 15.3767784	best: 15.3767784 (100)	total: 416ms	remaining: 20.2s
200:	learn: 14.4280201	test: 13.7416370	best: 13.7416370 (200)	total: 809ms	remaining: 19.3s
300:	learn: 13.3868554	test: 12.8201561	best: 12.8201561 (300)	total: 1.21s	remaining: 18.8s
400:	learn: 12.5585530	test: 12.0897086	best: 12.0897086 (400)	total: 1.61s	remaining: 18.5s
500:	learn: 11.9801125	test: 11.6084584	best: 11.6084584 (500)	total: 2.04s	remaining: 18.3s
600:	learn: 11.5434585	test: 11.2332092	best: 11.2332092 (600)	total: 2.45s	remaining: 17.9s
700:	learn: 11.1597477	test: 10.9095532	best: 10.9095532 (700)	total: 2.85s	remaining: 17.5s
800:	learn: 10.7995539	test: 10.6143143	best: 10.6143143 (800)	total: 3.25s	remaining: 17.1s
900:	learn: 10.4800340	test: 10.3496107	best: 10.3496107 (900)	total: 3.64s	remaining: 16.5s
1000:	learn: 10.2154405	test: 10.1419528	best: 10.1419528 (1000)	total: 4

CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=0]="ID_UOH62J0XHX": Cannot convert 'b'ID_UOH62J0XHX'' to float

In [ ]:
# train['pm2_5'].quantile(0.999)

def train_all(bl,features,SEED,k_folds):
    seedEverything(SEED)
    K_FOLDS = 5
    kf = KFold(n_splits = K_FOLDS, shuffle=False,random_state = None)
    y_predicted_keras,models_keras,y_oof_keras,oof_score_keras = train_keras(bl,train,features,target,test,kf,split_by)
    EARLY_STOPPING_ROUNDS = 30
    N_ESTIMATORS = 5000
    K_FOLDS = 2
    kf = KFold(n_splits = K_FOLDS, shuffle=False,random_state = None)
    y_predicted_lgb,models_lgb,y_oof_lgb,oof_score_lgb,features_importance_lgb = train_lgb(train,features,
                                                                                       target,test[features],
                                                                                       params['params_lgb'],kf,'pm2_5')
#     K_
    kf = KFold(n_splits = K_FOLDS, shuffle=True,random_state = SEED)

    y_predicted_cat,models_cat,y_oof_cat,oof_score_cat =train_catbo(train,,features,target,test,params['params_cat'],
                                                                    kf, 'pm2_5')
    
    y_predicted_xgb,models_xgb,y_oof_xgb,oof_score_xgb=train_xgb(train,features,target,test,
                                                             params['params_xgb'],kf,'pm2_5',num_iter=1000,es = 30,ve = 100)
    train['preds_lgb'] = y_oof_lgb                                                                                 
    train['preds_xgb'] = y_oof_xgb
    train['preds_cat'] = y_oof_cat                                                                                 
    train['preds_keras'] = y_oof_keras
    test['preds_lgb'] = y_predicted_lgb                                                                                 
    test['preds_xgb'] = y_predicted_xgb
    test['preds_cat'] = y_predicted_cat                                                                                 
    test['preds_keras'] = y_predicted_keras
    
    
                                                                                           
                                                                                           

In [ ]:
from keras import optimizers
def baseline_model(inp):
    # Create model here
    model = Sequential()
    model.add(Dense(100, input_dim = inp, activation = 'tanh')) # Rectified Linear Unit Activation Function
    model.add(Dropout(0.3))

#     model.add(Dense(32, activation = 'tanh'))
    model.add(Dense(8, activation = 'linear'))
    
    model.add(Dense(1, activation = 'linear')) # Softmax for multi-class classification
    # Compile model here
    model.compile(loss = 'mse', optimizer = tf.keras.optimizers.RMSprop(lr=0.025),
                  metrics = ['mse'],)
    return model

In [ ]:
 
SEED = 42
seedEverything(SEED)
K_FOLDS = 2
kf = KFold(n_splits = K_FOLDS, shuffle=False,random_state = None)
split_by = 'pm2_5'
y_predicted_keras,models_keras,y_oof_keras,oof_score_keras = train_keras(bl,train,features,train['pm2_5'],test,kf,split_by)

In [ ]:
preds_blend,_,w = models_to_blend()
# w

In [ ]:
preds_to_sub(test,preds_blend,'blend31')

In [57]:
EARLY_STOPPING_ROUNDS = 30
N_ESTIMATORS = 5000
K_FOLDS = 2
lgb_params = {
    "objective": "regression",
    "boosting_type": "gbdt",
    "learning_rate": 0.1,
    "n_jobs": 4,
    "seed": SEED,
    "max_depth": 13,
#     "num_leaves":10,
    "force_col_wise":True,
#     'min_data_in_leaf' : 5,
#     'feature_fraction':0.5,
#     'bagging_fraction':0.8,
#     "max_bin": 255,
#     'reg_lambda': 0.1,  # L1 regularization term on weights
#     'reg_lambda': 10,
    "metric": "MAE",
    "verbose": -1,
}
kf = KFold(n_splits = K_FOLDS, shuffle=False,random_state = None)
y_predicted_lgb,models_lgb,y_oof_lgb,oof_score_lgb,features_importance_lgb = train_lgb(train,features,
                                                                                       target,test,
                                                                                       lgb_params,kf,'pm2_5')

Shape of train_X : (9923, 69), test_X: (4254, 69), train_Y: (9923,)
fold 1 of 2
Best number of iterations for fold 1 is: 361
CV OOF Score for fold 1 is 7.766087489477163
fold 2 of 2
Best number of iterations for fold 2 is: 422
CV OOF Score for fold 2 is 8.511737588797063


In [55]:
target

'pm2_5'

In [ ]:
from catboost import Pool, CatBoostRegressor
params_cat = {'iterations':5000,
        'learning_rate':0.025,
#         'random_strength':0.1,
        'early_stopping_rounds':30, 
        'depth':3,
        'loss_function':'RMSE',
        'eval_metric':'MAE',
        'verbose' : 100,
#         'leaf_estimation_method':'Newton'
             }
kf = KFold(n_splits = K_FOLDS, shuffle=True,random_state = SEED)

y_predicted_cat,models_cat,y_oof_cat,oof_score_cat =train_catbo(train[features],
                                                                        train['pm2_5'],test[features],params_cat,
                                                                kf, 'pm2_5')

In [ ]:
params_xgb = {"objective":"reg:squarederror",'learning_rate': 0.1,
           'max_depth': 3}#, 'subsample': 0.9,
#           'colsample_bytree': 0.9}

y_predicted_xgb,models_xgb,y_oof_xgb,oof_score_xgb=train_xgb(train[features],
                                                                        train['pm2_5'],test[features],
                                                             params_xgb,kf,'pm2_5',num_iter=1000,es = 30,ve = 100)

In [ ]:
def models_to_blend():
    w_cat = cust_score(y_oof_cat).sort_values(by = 'err', ascending = False)['err'].iloc[:4].mean()
    w_lgb = cust_score(y_oof_lgb).sort_values(by = 'err', ascending = False)['err'].iloc[:4].mean()
    w_xgb = cust_score(y_oof_xgb).sort_values(by = 'err', ascending = False)['err'].iloc[:4].mean()
    w_keras = cust_score(y_oof_keras).sort_values(by = 'err', ascending = False)['err'].iloc[:4].mean()
    weigths = [1/w_cat,1/w_lgb,1/w_xgb,1/w_keras]
    blend_preds = np.average(np.vstack([y_predicted_cat,y_predicted_xgb,y_predicted_cat,y_predicted_keras.squeeze()]),axis = 0,weights=weigths)
    blend_preds_train = np.average(np.vstack([y_oof_cat,y_oof_xgb,y_oof_cat,y_oof_keras.squeeze()]),axis = 0,weights=weigths)
    w_blend = cust_score(blend_preds_train).sort_values(by = 'err', ascending = False)['err'].iloc[:4].mean()
    weights = [w_cat,w_lgb,w_xgb,w_keras,w_blend]
    print("mae of cat : {}.-------- worst {}".format(metrics.mean_absolute_error(train['pm2_5'],y_oof_cat).round(3),w_cat.round(3)))
    print("mae of lgb : {}..-------- worst {}".format(metrics.mean_absolute_error(train['pm2_5'],y_oof_lgb).round(3),w_lgb.round(3)))
    print("mae of xgb : {}..-------- worst {}".format(metrics.mean_absolute_error(train['pm2_5'],y_oof_xgb).round(3),w_xgb.round(3)))
    print("mae of keras : {}..-------- worst {}".format(metrics.mean_absolute_error(train['pm2_5'],y_oof_keras).round(3),w_keras.round(3)))
    print("mae of blend : {}..-------- worst {}".format(metrics.mean_absolute_error(train['pm2_5'],blend_preds_train).round(3),w_blend.round(3)))
          
    return blend_preds,blend_preds_train,weights
    

In [ ]:
_,w = models_to_blend()

In [ ]:
w

In [ ]:
np.vstack([y_predicted_lgb,y_predicted_xgb]).shape

In [ ]:
# y_predicted_xgb

In [ ]:
np.average(np.vstack([y_predicted_lgb,y_predicted_xgb]),axis = 0,weights=[10,1])

In [ ]:
preds_to_sub(test,y_predicted_lgb+,'lgb_scaled3')

In [ ]:
import optuna
SEED = 42
K_FOLDS = 4
target = 'pm2_5'
def objective(trial):
    params = {
                #'iterations' : 10000, replaced by early stopping
                'eval_metric': 'RMSE',
                'verbose' : 500,
                'use_best_model': True,
                'random_seed' : SEED,
                'learning_rate' :trial.suggest_loguniform('learning_rate', 0.01, 0.3),
                "depth": trial.suggest_int("depth", 1, 9),
                'l2_leaf_reg' :trial.suggest_loguniform('l2_leaf_reg', 1e-8, 10),
                'random_strength' : trial.suggest_loguniform('random_strength', 0.1, 10),
                'grow_policy':trial.suggest_categorical ('grow_policy', ['Lossguide','SymmetricTree']),
                'max_bin': trial.suggest_int("max_bin", 20, 500),
                'min_data_in_leaf':trial.suggest_int('min_data_in_leaf', 1, 100),
                "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli"])
            }
    
    if params['grow_policy'] == 'Lossguide':
        params['max_leaves']:trial.suggest_int('max_leaves', 1, 20)
    if params["bootstrap_type"] == "Bayesian":
        params["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif params["bootstrap_type"] == "Bernoulli":
        params["subsample"] = trial.suggest_float("subsample", 0.1, 1)
        
    
    kf = KFold(n_splits = K_FOLDS, shuffle=True,random_state = SEED)


    y_predicted_cat,models_cat,y_oof_cat,oof_score_cat =train_catbo(train,features,target,test,params,
                                                                    kf=kf, split_by = 'pm2_5')
    
        
    return cust_score(y_oof_cat).sort_values(by = 'err', ascending = False)['err'].iloc[:4].mean()

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

In [27]:
# features
# study.best_params
# {'learning_rate': 0.09336356367527626,
#  'depth': 8,
#  'l2_leaf_reg': 0.00033354798569319656,
#  'random_strength': 10.395143072706773,
#  'grow_policy': 'SymmetricTree',
#  'max_bin': 111,
#  'min_data_in_leaf': 94,
#  'bootstrap_type': 'Bernoulli',
#  'subsample': 0.4351151265075187}### params with MAE, metric
cat_b_params = {'learning_rate': 0.07574755563809993, 'depth': 6, 'l2_leaf_reg': 0.39323612155779863, 'random_strength': 0.20259568870497338, 'grow_policy': 'Lossguide', 'max_bin': 67, 'min_data_in_leaf': 10, 'bootstrap_type': 'Bernoulli', 'subsample': 0.9244602042319705}

{'learning_rate': 0.09336356367527626,
 'depth': 8,
 'l2_leaf_reg': 0.00033354798569319656,
 'random_strength': 10.395143072706773,
 'grow_policy': 'SymmetricTree',
 'max_bin': 111,
 'min_data_in_leaf': 94,
 'bootstrap_type': 'Bernoulli',
 'subsample': 0.4351151265075187}

In [ ]:
# study.direction
K_FOLDS = 5
# kf = KFold(n_splits=N_split)
SEED = 43
import optuna

def objective(trial):
   
    params = { 'objective': 'reg:squarederror',
              'eval_metric': 'mae',
#             'tree_method': 'hist',
            'grow_policy' : trial.suggest_categorical ('grow_policy', ['lossguide','depthwise']),
            'learning_rate':trial.suggest_loguniform('learning_rate', 0.01, 0.2),
            'max_depth': trial.suggest_int('max_depth', 3, 20),# a virer avec'depthwise'
#             'reg_alpha': trial.suggest_loguniform('reg_alpha', 1, 10),
#             'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-15, 10.0),
            'max_delta_step':trial.suggest_int('max_delta_step', 1, 10),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
            'colsample_bytree':trial.suggest_loguniform('colsample_bytree', 0.4, 1.0),
            'subsample': trial.suggest_loguniform('subsample', 0.4, 1.0),
            'seed':1
                }
    if params['grow_policy'] == 'lossguide':
        params['max_leaves'] = trial.suggest_int('max_leaves', 1, 100)   
        
    kf = KFold(n_splits = K_FOLDS, shuffle=True,random_state = SEED)

    y_predicted_xgb,models_xgb,y_oof_xgb,oof_score_xgb=train_xgb(train,features,target,test,
                                                             params,kf,'pm2_5',num_iter=1000,es = 30,ve = 500)
        
    return cust_score(y_oof_xgb).sort_values(by = 'err', ascending = False)['err'].iloc[:4].mean()


study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

In [ ]:
# 4 folds
# xgb_params = {'grow_policy': 'depthwise', 'learning_rate': 0.10769618165955801, 'max_depth': 14, 'max_delta_step': 2, 'min_child_weight': 132, 'colsample_bytree': 0.6353119293029151, 'subsample': 0.6254457606715289,'tree_method': 'hist'}
# xgb_params = {'grow_policy': 'lossguide', 'learning_rate': 0.03982304720613181, 'max_depth': 12, 'max_delta_step': 10, 'min_child_weight': 133, 'colsample_bytree': 0.8709673399493595, 'subsample': 0.7129355859703879, 'max_leaves': 34}

In [ ]:
# N_split = 2
# kf = KFold(n_splits=N_split)
# EARLY_STOPPING_ROUNDS = 30
N_ESTIMATORS = 5000
K_FOLDS = 3
import optuna

def objective(trial):
    params = {
            'objective': 'regression',
            'metric': 'mae',
            'n_estimators':5000,
            'verbose':-1,
            'learning_rate': trial.suggest_float("learning_rate", 0.04,0.4),
            'num_leaves': trial.suggest_int('num_leaves', 10, 100),
            'colsample_bytree':trial.suggest_float("colsample", 0.1,0.3),
            'subsample': trial.suggest_float("subsample", 0.1,0.3),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'min_child_samples': trial.suggest_int('min_child_samples', 3, 2000),
            'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 10.0),
            'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),
            'cat_smooth':trial.suggest_int('cat_smooth', 1, 100)
            }
    
    kf = KFold(n_splits = K_FOLDS, shuffle=False,random_state = None)
    y_predicted_lgb,models_lgb,y_oof_lgb,oof_score_lgb,features_importance_lgb = train_lgb(train,features,
                                                                                  target,test,params = params,kf = kf,split_by = 'pm2_5') 
    return cust_score(y_oof_lgb).sort_values(by = 'err', ascending = False)['err'].iloc[:4].mean()


study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

[I 2022-09-21 15:15:11,231] A new study created in memory with name: no-name-efd03da3-7424-4b7c-be04-8354faea1eab


Shape of train_X : (9923, 69), test_X: (4254, 69), train_Y: (9923,)
fold 1 of 3
Best number of iterations for fold 1 is: 0
CV OOF Score for fold 1 is 9.158008505782536
fold 2 of 3
Best number of iterations for fold 2 is: 0
CV OOF Score for fold 2 is 9.304642945031127
fold 3 of 3
Best number of iterations for fold 3 is: 0
CV OOF Score for fold 3 is 9.412341891165092


[I 2022-09-21 15:15:23,402] Trial 0 finished with value: 15.972997253821 and parameters: {'learning_rate': 0.3939723623264472, 'num_leaves': 50, 'colsample': 0.10607282907856037, 'subsample': 0.23288542443763074, 'max_depth': 3, 'min_child_samples': 764, 'reg_alpha': 0.0028214770453633647, 'reg_lambda': 3.5201438870240126e-06, 'cat_smooth': 39}. Best is trial 0 with value: 15.972997253821.


Shape of train_X : (9923, 69), test_X: (4254, 69), train_Y: (9923,)
fold 1 of 3
Best number of iterations for fold 1 is: 0
CV OOF Score for fold 1 is 9.767121886379101
fold 2 of 3
Best number of iterations for fold 2 is: 0
CV OOF Score for fold 2 is 10.102051245546077
fold 3 of 3
Best number of iterations for fold 3 is: 0
CV OOF Score for fold 3 is 10.188515104275174


[I 2022-09-21 15:15:35,999] Trial 1 finished with value: 17.120524641990993 and parameters: {'learning_rate': 0.2843324044679964, 'num_leaves': 22, 'colsample': 0.2114746501308415, 'subsample': 0.24250409880089902, 'max_depth': 3, 'min_child_samples': 1655, 'reg_alpha': 0.040621176561131116, 'reg_lambda': 3.8109214519332717e-06, 'cat_smooth': 57}. Best is trial 0 with value: 15.972997253821.


...

Shape of train_X : (9923, 69), test_X: (4254, 69), train_Y: (9923,)
fold 1 of 3
Best number of iterations for fold 1 is: 0
CV OOF Score for fold 1 is 8.321591217639735
fold 2 of 3
Best number of iterations for fold 2 is: 0
CV OOF Score for fold 2 is 8.451166817967477
fold 3 of 3
Best number of iterations for fold 3 is: 0
CV OOF Score for fold 3 is 8.59146847455155


[I 2022-09-21 15:20:44,328] Trial 14 finished with value: 15.595327883453734 and parameters: {'learning_rate': 0.11010913818085803, 'num_leaves': 62, 'colsample': 0.295441165093243, 'subsample': 0.26525296791364483, 'max_depth': 10, 'min_child_samples': 370, 'reg_alpha': 8.757835168267108e-06, 'reg_lambda': 0.5140109899302303, 'cat_smooth': 79}. Best is trial 10 with value: 14.573589497058597.


Shape of train_X : (9923, 69), test_X: (4254, 69), train_Y: (9923,)
fold 1 of 3
Best number of iterations for fold 1 is: 0
CV OOF Score for fold 1 is 8.661202520083748
fold 2 of 3
Best number of iterations for fold 2 is: 0
CV OOF Score for fold 2 is 8.845501984215995
fold 3 of 3
Best number of iterations for fold 3 is: 0
CV OOF Score for fold 3 is 8.87614110502353


[I 2022-09-21 15:21:02,173] Trial 15 finished with value: 15.525848036802953 and parameters: {'learning_rate': 0.19232277338990345, 'num_leaves': 10, 'colsample': 0.17281721447809675, 'subsample': 0.20784182899265688, 'max_depth': 5, 'min_child_samples': 315, 'reg_alpha': 1.3750029851930742e-08, 'reg_lambda': 0.03229045960477008, 'cat_smooth': 85}. Best is trial 10 with value: 14.573589497058597.


Shape of train_X : (9923, 69), test_X: (4254, 69), train_Y: (9923,)
fold 1 of 3
